# Decision Tree Implementation

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
class Node:

    def __init__(self, left=None, right=None, feature=None, split_value=None, data=None):
        self.left = left
        self.right = right
        self.feature = feature
        self.split_value = split_value
        self.data = data

In [3]:
class DecisionTreeClassifier:

    def __init__(self, min_values=10):
        self.root = None
        self.min_values = min_values

    def entropy(self, y):
        elements, counts = np.unique(y, return_counts=True)
        probabilities = [i / len(y) for i in counts]
        entropy = np.sum([-i * np.log2(i) for i in probabilities])
        return entropy

    def information_gain(self, y, left_side, right_side):
        parent_entropy = self.entropy(y)
        average_child_entropy = len(left_side) / len(y) * self.entropy(y[left_side]) + len(right_side) / len(y) * self.entropy(y[right_side])
        IG = parent_entropy - average_child_entropy
        return IG

    def fit(self, X, y):
        n_classes = len(np.unique(y))

        # stopping conditions
        if n_classes == 1 or X.shape[0] <= self.min_values:
            leaf_node = lambda x: np.bincount(x).argmax()
            return Node(data=leaf_node(y))

        # search all data points to find the feature and split value with the highest information gain
        features = [i for i in range(0, X.shape[1])]
        best_feature, best_split_value, gain = 0, 0, 0
        for feature in features:
            X_column = X[:, feature]
            split_values = np.unique(X_column)
            for split_value in split_values:
                left_side, right_side = self.binary_separation(X_column, split_value)
                information_gain = self.information_gain(y, left_side, right_side)
                if information_gain > gain:
                    gain = information_gain
                    best_feature = feature
                    best_split_value = split_value

        # grow tree recursively
        left_side, right_side = self.binary_separation(X[:, best_feature], best_split_value)
        left = self.fit(X[left_side, :], y[left_side])
        right = self.fit(X[right_side, :], y[right_side])
        self.root = Node(left, right, best_feature, best_split_value)
        return self.root
    
    def binary_separation(self, X_column, split_value):
        left_side = np.where(X_column <= split_value)
        right_side = np.where(X_column > split_value)
        return left_side[0], right_side[0]

    def predict(self, X):

        def predict_for(i, node):
            if node.left is None and node.right is None:
                return node.data
            elif i[node.feature] <= node.split_value:
                return predict_for(i, node.left)
            else:
                return predict_for(i, node.right)

        return np.array([predict_for(i, self.root) for i in X])

    def accuracy(self, y_true, y_prediction):
        return np.sum(y_true == y_prediction) / len(y_true)

# Random Forest Implementation

In [4]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [7]:
class RandomForest:

    def __init__(self, n_trees = 50):
        self.n_trees = n_trees

    def bootstrap(self, X, y):
        indexes = np.random.choice(len(X), len(X), replace=True)
        return X[indexes, :], y[indexes]

    def fit(self, X, y):
        self.trees = list()
        tree = DecisionTreeClassifier()
        for i in range(self.n_trees):
            X_sample, y_sample = self.bootstrap(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees]).T
        y_prediction = [np.bincount(p).argmax() for p in predictions]
        return np.array(y_prediction)

    def accuracy(self, y_true, y_prediction):
        return np.sum(y_true == y_prediction) / len(y_true)

In [13]:
if __name__ == "__main__":

    data = datasets.load_breast_cancer()
    X = data.data
    y = data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    rf = RandomForest(20)
    rf.fit(X_train, y_train)
    y_prediction = rf.predict(X_test)
    accuracy = rf.accuracy(y_test, y_prediction)
    print("Accuracy rate of Random Forest Implementation:", accuracy)

Accuracy rate of Random Forest Implementation: 0.9300699300699301
